In [44]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os
import random
import cv2
import matplotlib
from sklearn.model_selection import train_test_split
from skimage.filters import threshold_otsu
import torchmetrics
import pickle
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torchvision.models as models
import torchvision
from torchvision import transforms
import torch
import torch.nn as nn
from torch.optim import Adam
from torch import LongTensor as LongTensor
from torch import FloatTensor as FloatTensor
import cv2
from collections import Counter
from sklearn.model_selection import StratifiedKFold

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

## Parameters

In [3]:
batch_size = 16
number_of_tiles_per_bag = 180
number_of_folds = 20
n_epoch = 100
early_stopping = 10
learning_rate = 0.0001

## Load data

In [4]:
train_set_path = "./Data/raw_data/train.csv"
train_tiles_encoding_folder = "./Data/processed_data/train_tiles_encoding/"

test_set_path = "./Data/raw_data/test.csv"
test_tiles_encoding_folder = "./Data/processed_data/test_tiles_encoding/"

In [5]:
initial_train_set = pd.read_csv(train_set_path)
test_set = pd.read_csv(test_set_path)

In [6]:
## keep only if enough tiles retrieve
to_remove = []
for idx in range(initial_train_set.shape[0]):
  value = initial_train_set.iloc[idx]
  image_id = value.image_id
  # compute file
  image_embeddings_path = train_tiles_encoding_folder+image_id+'.pkl'

  # open embeddings  dict
  embeddings_dict = pickle.load(open(image_embeddings_path, 'rb'))
  if len(embeddings_dict) < number_of_tiles_per_bag:
    to_remove.append(image_id)

initial_train_set = initial_train_set.query('image_id not in @to_remove')

## Prepare Data

In [7]:
kf = StratifiedKFold(n_splits=number_of_folds)
folds_dict = {}
KFold(random_state=None, shuffle=True)
for i, (train_index, validation_index) in enumerate(kf.split(initial_train_set.drop(columns = ['isup_grade']), initial_train_set[['isup_grade']])):
    folds_dict[i] = (train_index, validation_index)

## Define epoch training

In [8]:
# 1 epoch training

def train_one_epoch(model, trainloader, validationloader, optimizer, device): 
    losses = []

    val_auroc = torchmetrics.AUROC(num_classes = 6)
    val_f1 = torchmetrics.F1Score(num_classes = 6)
    train_f1 = torchmetrics.F1Score(num_classes = 6)
    best_validation_f1 = -np.inf

    ### traning 
    model.train()
    for (features, target) in tqdm(trainloader):
        features, target = features.to(device), target.to(device)

        optimizer.zero_grad()
        predictions = model(features)
        predicted_classes = torch.argmax(predictions, dim=1)
      
        criterion = nn.CrossEntropyLoss()
        loss = criterion(predictions, target)
        losses.append(float(loss))
        loss.backward()
        optimizer.step()
        f1_train = train_f1(predicted_classes.cpu(), target.cpu())

    ### model evaluation 
    model.eval()
    with torch.no_grad():
      for (features, target) in (validationloader):
          features, target = features.to(device), target.to(device)

          predictions = model(features)
          predicted_classes = torch.argmax(predictions, dim=1)
          
          validation_auroc = val_auroc(predictions.cpu().detach(), target.cpu().detach())
          f1_val = val_f1(predicted_classes.cpu().detach(), target.cpu().detach())


    val_f1_final = val_f1.compute()    
    train_f1_final = train_f1.compute()  
    val_auroc_final = val_auroc.compute()
    print('average train loss: ', np.mean(losses))
    print('validation f1: ', val_f1_final)
    print('train f1: ', train_f1_final)
    print('validation auroc: ', val_auroc_final)

    return val_auroc_final

In [9]:
def train_model(model, trainloader, validationloader, optimizer, device, n_epoch, checkpoint_path, early_stopping):
    model.to(device)

    
    best_roc = -np.inf
    previous_roc = -np.inf
    counter = 0
    for epoch in range(0, n_epoch):
        print(f"epoch {epoch+1}/{n_epoch}")
        roc = train_one_epoch(model , trainloader, validationloader, optimizer, device)
        if roc >= best_roc:
            best_roc = roc
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'roc': roc,
            }, checkpoint_path)

            print('New best model saved !')

        if roc <= best_roc:
            print('no iprovement')
            counter += 1
        
        else:
            counter = 0

        if counter == early_stopping:
            print('early stopping !')
            break 
    model.cpu() # remove model from gpu
    return



## Define data sets


In [10]:
class EmbeddingDatasetNoPosition(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, df, dir, bag_size = 180):
        """
        Args:
            df (dataframe): Path to the csv file with annotations.
            dir (string): Directory with tiles images embeddings
        """
        self.df = df
        self.dir = dir
        self.bag_size = bag_size

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        value = self.df.iloc[idx]
        image_id = value.image_id
        # compute file
        image_embeddings_path = self.dir+image_id+'.pkl'

        # open embeddings  dict
        embeddings_dict = pickle.load(open(image_embeddings_path, 'rb'))

        bag = None
        embedding_list = np.array(list(embeddings_dict.values()))
        np.random.shuffle(embedding_list)
        bag = np.vstack(embedding_list[:self.bag_size])

        bag = torch.Tensor(bag)
        # compute label
        label = torch.tensor(value.isup_grade)

        return bag, label

## Define Models

In [11]:
class ModelVanilla(nn.Module):
    def __init__(self, number_of_embeddings = 180, output_shape = 6, embedding_shape = 1280):
        super(ModelVanilla, self).__init__()
        
        # to encode each bag
        self.aggregator = torch.nn.AvgPool1d(number_of_embeddings)


        self.classifier = nn.Sequential(
                        nn.Linear(embedding_shape, int(embedding_shape/2)),
                        nn.ReLU(),
                        torch.nn.BatchNorm1d(int(embedding_shape/2)),
                        nn.Dropout(0.25),
                        nn.Linear(int(embedding_shape/2), int(embedding_shape/4)),
                        nn.ReLU(),
                        nn.Dropout(0.25),
                        torch.nn.BatchNorm1d(int(embedding_shape/4)),
                        nn.Linear(int(embedding_shape/4), output_shape)
                        )
        
        
        self.softmax = nn.Softmax(dim = 1)


        
    def forward(self, bag):
      bag = torch.transpose(bag, 1, 2)
      wsi_descriptor = self.aggregator(bag).squeeze(-1)
      out = self.classifier(wsi_descriptor)
      out = self.softmax(out)
      return out

In [10]:
class AttentionModel(nn.Module):
    def __init__(self, number_of_embeddings = 180, output_shape = 6, embedding_shape = 1280):
        super(AttentionModel, self).__init__()
        
        # to compute attenation map for aggregation
        self.attention = nn.Sequential(
            nn.Linear(embedding_shape, int(embedding_shape/4)),
            nn.Tanh(),
            nn.Linear(int(embedding_shape/4), 1)
            )


        self.classifier = nn.Sequential(
                        nn.Linear(embedding_shape, int(embedding_shape/2)),
                        nn.ReLU(),
                        torch.nn.BatchNorm1d(int(embedding_shape/2)),
                        nn.Dropout(0.25),
                        nn.Linear(int(embedding_shape/2), int(embedding_shape/4)),
                        nn.ReLU(),
                        torch.nn.BatchNorm1d(int(embedding_shape/4)),
                        nn.Dropout(0.25),
                        nn.Linear(int(embedding_shape/4), output_shape)
        )
        
        
        self.softmax = nn.Softmax(dim = 1)


        
    def forward(self, bag):
      # compute attention map
      attention_map = self.attention(bag)
      attention_map = torch.transpose(attention_map, 2, 1) # KxN
      attention_map = self.softmax(attention_map)
      # apply attention map
      M = torch.matmul(attention_map, bag).squeeze(1)
      # classification
      out = self.classifier(M)
      out = self.softmax(out)
      return out

In [9]:
class ModelChowder(nn.Module):
    def __init__(self, number_of_embeddings = 180, output_shape = 6, embedding_shape = 1280, R = 5):
        super(ModelChowder, self).__init__()
        
        # to encode each bag
        self.R = R
        self.conv = torch.nn.Conv1d(number_of_embeddings,number_of_embeddings,kernel_size =embedding_shape)
        
        self.classifier = nn.Sequential(
                        nn.Linear(2*self.R, 200),
                        nn.Sigmoid(),
                        torch.nn.BatchNorm1d(200),
                        nn.Dropout(0.25),
                        nn.Linear(200, 100),
                        nn.Sigmoid(),
                        torch.nn.BatchNorm1d(100),
                        nn.Dropout(0.25),
                        nn.Linear(100, output_shape)
                        )
        
        
        self.softmax = nn.Softmax(dim = 1)


        
    def forward(self, bag):
      post_conv = self.conv(bag).squeeze(-1)
      post_conv = torch.sort(post_conv)[0]

      max_values = post_conv[:,-self.R:]
      min_values = post_conv[:,:self.R]

      wsi_descriptor = torch.cat((max_values,min_values), 1)



      out = self.classifier(wsi_descriptor)
      out = self.softmax(out)
      return out

## Train models

In [14]:
for fold_number, (train_index, val_index) in folds_dict.items():
    print(f"train fold: {fold_number}")

    # split train/val
    train_set, validation_set = initial_train_set.iloc[train_index], initial_train_set.iloc[val_index]

    # create data sets
    train_set = EmbeddingDatasetNoPosition(train_set, train_tiles_encoding_folder, bag_size = number_of_tiles_per_bag)
    validation_set = EmbeddingDatasetNoPosition(validation_set, train_tiles_encoding_folder, bag_size = number_of_tiles_per_bag)

    # create loaders
    trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last = True)
    validationloader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=True, drop_last = True)

    # train models

    # Chowder model
    print("start training Chowder model")
    model = ModelChowder(number_of_embeddings = number_of_tiles_per_bag)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate )
    checkpoint_path_chowder = f"./Data/checkpoints/model_chowder_fold_{fold_number+1}"
    train_model(model, trainloader, validationloader, optimizer, device, n_epoch, checkpoint_path_chowder, early_stopping)

    # Vanilla model
    print("start training Vanilla model")
    model = ModelVanilla(number_of_embeddings = number_of_tiles_per_bag)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate )
    checkpoint_path_vanilla = f"./Data/checkpoints/model_vanilla_{fold_number+1}"
    best_model = train_model(model, trainloader, validationloader, optimizer, device, n_epoch, checkpoint_path_vanilla, early_stopping)

    # Attention model
    print("start training Attention model")
    model = AttentionModel(number_of_embeddings = number_of_tiles_per_bag)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate )
    checkpoint_path_attention = f"./Data/checkpoints/model_attention_{fold_number+1}"
    best_model = train_model(model, trainloader, validationloader, optimizer, device, n_epoch, checkpoint_path_attention, early_stopping)

train fold: 0
start training Chowder model


c:\Users\33631\Documents\Etudes\centrale 3A\Deep Learning for medical Imaging\prostate_cancer_detection\venv\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


epoch 1/100


100%|██████████| 20/20 [00:08<00:00,  2.45it/s]


average train loss:  1.771587324142456
validation f1:  tensor(0.0625)
train f1:  tensor(0.2188)
validation auroc:  tensor(0.5889)
New best model saved !
no iprovement
epoch 2/100


...

average train loss:  1.525807785987854
validation f1:  tensor(0.3125)
train f1:  tensor(0.5562)
validation auroc:  tensor(0.7553)
no iprovement
epoch 9/100


100%|██████████| 20/20 [00:01<00:00, 10.65it/s]


average train loss:  1.494756019115448
validation f1:  tensor(0.5000)
train f1:  tensor(0.6062)
validation auroc:  tensor(0.7367)
no iprovement
epoch 10/100


100%|██████████| 20/20 [00:02<00:00,  9.72it/s]


average train loss:  1.4862616062164307
validation f1:  tensor(0.3750)
train f1:  tensor(0.6031)
validation auroc:  tensor(0.6794)
no iprovement
early stopping !


# Prediction

In [12]:
model_chowder = ModelChowder(number_of_embeddings = number_of_tiles_per_bag)
model_vanilla = ModelVanilla(number_of_embeddings = number_of_tiles_per_bag)
model_attention = AttentionModel(number_of_embeddings = number_of_tiles_per_bag)

In [18]:
final_df = pd.DataFrame(columns = ['Id','Predicted'])
for i in tqdm(range(test_set.shape[0]), position = 0):

  value = test_set.iloc[i]
  id = value.image_id
  # compute file
  image_embeddings_path = test_tiles_encoding_folder+id+'.pkl'
  # open embeddings  dict
  embeddings_dict = pickle.load(open(image_embeddings_path, 'rb'))
  bag = None
  embedding_list = np.array(list(embeddings_dict.values()))
  np.random.shuffle(embedding_list)
  bag = np.vstack(embedding_list[:number_of_tiles_per_bag])
  bag = torch.Tensor(bag).unsqueeze(0)

  final_pred = [0 for i in range(6)]
  for fold_number in range(number_of_folds):
    # open checkpoints
    checkpoint_chowder = torch.load(f"./Data/checkpoints/model_chowder_fold_{fold_number+1}")
    checkpoint_vanilla = torch.load(f"./Data/checkpoints/model_vanilla_{fold_number+1}")
    checkpoint_attention = torch.load(f"./Data/checkpoints/model_attention_{fold_number+1}")

    # load weights
    model_chowder.load_state_dict(checkpoint_chowder['model_state_dict']) 
    model_vanilla.load_state_dict(checkpoint_vanilla['model_state_dict']) 
    model_attention.load_state_dict(checkpoint_attention['model_state_dict']) 


    # get predictions
    model_chowder.eval()
    pred_chowder = np.int32(np.argmax(model_chowder(bag).detach().cpu().numpy()))

    model_vanilla.eval()
    pred_vanilla = np.int32(np.argmax(model_vanilla(bag).detach().cpu().numpy()))


    model_attention.eval()
    pred_attention = np.int32(np.argmax(model_attention(bag).detach().cpu().numpy()))


    # majority voting of the current fold
    pred_fold = [0 for i in range(6)]
    pred_fold[pred_chowder] += 1
    pred_fold[pred_vanilla] += 1
    pred_fold[pred_attention] += 1
    pred_current_fold = np.int32(np.argmax(np.array(pred_fold)))


    final_pred[pred_current_fold] += 1
  

  # majority voting accross folds
  current_line = pd.DataFrame([[id, np.int32(np.argmax(np.array(final_pred)))]], columns = ['Id','Predicted'])
  final_df = pd.concat([final_df, current_line], axis = 0)


final_df.to_csv("/content/drive/MyDrive/centrale_3A/Deep Learning for medical imaging/final_preds.csv", index = False)


  0%|          | 0/86 [00:20<?, ?it/s]


KeyboardInterrupt: 

In [25]:
final_df = pd.DataFrame(columns = ['Id','Predicted'])
total_predictions = []
for fold_number in range(number_of_folds):
    print(f"current fold: {fold_number+1}")
    # open checkpoints
    checkpoint_chowder = torch.load(f"./Data/checkpoints/model_chowder_fold_{fold_number+1}")
    checkpoint_vanilla = torch.load(f"./Data/checkpoints/model_vanilla_{fold_number+1}")
    checkpoint_attention = torch.load(f"./Data/checkpoints/model_attention_{fold_number+1}")

    # load weights
    model_chowder.load_state_dict(checkpoint_chowder['model_state_dict']) 
    model_vanilla.load_state_dict(checkpoint_vanilla['model_state_dict']) 
    model_attention.load_state_dict(checkpoint_attention['model_state_dict']) 

    predictions_fold= []
    for i in tqdm(range(test_set.shape[0]), position = 0):
        value = test_set.iloc[i]
        id = value.image_id
        # compute file
        image_embeddings_path = test_tiles_encoding_folder+id+'.pkl'
        # open embeddings  dict
        embeddings_dict = pickle.load(open(image_embeddings_path, 'rb'))
        bag = None
        embedding_list = np.array(list(embeddings_dict.values()))
        np.random.shuffle(embedding_list)
        bag = np.vstack(embedding_list[:number_of_tiles_per_bag])
        bag = torch.Tensor(bag).unsqueeze(0)


        # get predictions
        model_chowder.eval()
        pred_chowder = np.int32(np.argmax(model_chowder(bag).detach().cpu().numpy()))

        model_vanilla.eval()
        pred_vanilla = np.int32(np.argmax(model_vanilla(bag).detach().cpu().numpy()))

        model_attention.eval()
        pred_attention = np.int32(np.argmax(model_attention(bag).detach().cpu().numpy()))

        # majority voting of the current fold
        pred_fold = [0 for i in range(6)]
        pred_fold[pred_chowder] += 1
        pred_fold[pred_vanilla] += 1
        pred_fold[pred_attention] += 1
        pred_current_fold = np.int32(np.argmax(np.array(pred_fold)))

        predictions_fold.append(pred_current_fold)

    total_predictions.append(predictions_fold)

current fold: 1


100%|██████████| 86/86 [00:09<00:00,  8.86it/s]


current fold: 2


100%|██████████| 86/86 [00:07<00:00, 10.77it/s]


current fold: 3


...

100%|██████████| 86/86 [00:10<00:00,  8.40it/s]


current fold: 19


100%|██████████| 86/86 [00:11<00:00,  7.41it/s]


current fold: 20


100%|██████████| 86/86 [00:08<00:00,  9.82it/s]


In [58]:
# max voiting between folds
a = np.array(total_predictions)
final_prediction = []
for i in range(a.shape[1]):
    count_dict = Counter(a[:,i])
    pred  = max(count_dict, key=count_dict.get)
    final_prediction.append(pred)


In [65]:
# create final data frame
final_df = pd.DataFrame()
final_df['Id'] = list(test_set.image_id)
final_df['Predicted'] = final_prediction

final_df.to_csv("final_preds.csv", index = False)